In [ ]:
import anndata
import scanpy as sc
import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
meta = pd.read_csv("../data/samplesheet_Grabherr.csv")

In [ ]:
meta = meta.drop(axis="columns", labels=["fastq_1", "fastq_2"])

In [ ]:
meta

In [ ]:
adatas = []
for sample in meta.to_dict(orient="records"):
    tmp_adata = sc.read_10x_h5(
        f"/data/projects/2021/Grabherr-scRNAseq-mouse/30_nfcore_scrnaseq_v2-0-0/cellranger/sample-{sample['sample']}/outs/filtered_feature_bc_matrix.h5"
    )
    tmp_adata.var_names_make_unique()
    assert tmp_adata.obs_names.is_unique
    tmp_adata.obs = tmp_adata.obs.assign(**sample)
    adatas.append(tmp_adata)
adata = anndata.concat(adatas, index_unique="_")

In [ ]:
adata

In [ ]:
pd_meta_dat = pd.read_csv('../data/samplesheet_Grabherr.csv',
                          index_col='sample')
pd_meta_dat = pd_meta_dat.drop(axis="columns", labels=["fastq_1", "fastq_2"])
for f, sample in zip(pd_meta_dat.index, pd_meta_dat.to_dict(orient="records")):
    print(sample)

In [ ]:
path_list = []
adatas_list = []
n_barcodes_dict = {}
for ind, sample in zip(pd_meta_dat.index, pd_meta_dat.to_dict(orient="records")):

    pathi = f'/data/projects/2021/Grabherr-scRNAseq-mouse/30_nfcore_scrnaseq_v2-0-0/cellranger/sample-{ind}/outs/filtered_feature_bc_matrix.h5'
    path_list.append(pathi)
    dat_l = sc.read_10x_h5(pathi)
    dat_l.var_names_make_unique()
    assert dat_l.obs_names.is_unique
    dat_l.obs = dat_l.obs.assign(**sample)
    adatas_list.append(dat_l)
    
    n_barcodes_dict[sample['internal_id']] = np.shape(dat_l)[0]

adata_v = anndata.concat(adatas_list, index_unique="_")

In [ ]:
for int_id in adata_v.obs.internal_id.unique():
    adata_v.obs.loc[adata_v.obs.internal_id == int_id, 'n_barcodes'] = str(n_barcodes_dict[int_id])

In [ ]:
adata_v.obs['label'] = adata_v.obs['internal_id'] + '\nn=' + adata_v.obs['n_barcodes'].astype(str)

In [ ]:
adata_v.obs.n_barcodes.unique().astype(int).sum()

In [ ]:
adata_v.var['MT'] = adata_v.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata_v, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata_v.obs.head()

In [ ]:
sc.pl.violin(
    adata_v, 
    [
     'n_genes_by_counts', 
     'total_counts', 
     'pct_counts_MT'
     ],
    multi_panel=True
)

In [ ]:
sc.pl.scatter(adata_v, "total_counts", "n_genes_by_counts")

In [ ]:
# Quality control - plot QC metrics
#Sample quality plots
sns.set(rc={"figure.figsize":(30,10)})
sns.set(font_scale=2)
t1 = sc.pl.violin(adata_v, 'total_counts', groupby='label', size=2, log=True, cut=0)
#plt.savefig('blub.pdf')
t2 = sc.pl.violin(adata_v, 'pct_counts_MT', groupby='label')

In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(adata_v, 'total_counts', 'n_genes_by_counts', color='pct_counts_MT')
p2 = sc.pl.scatter(adata_v[adata_v.obs['total_counts']<10000], 'total_counts', 'n_genes_by_counts', color='pct_counts_MT')

In [ ]:
#Thresholding decision: counts
p3 = sns.distplot(adata_v.obs['total_counts'], kde=False, bins=60)
plt.show()

p4 = sns.distplot(adata_v.obs['total_counts'][adata_v.obs['total_counts']<1000], kde=False, bins=100)
plt.show()

p5 = sns.distplot(adata_v.obs['total_counts'][adata_v.obs['total_counts']>1000], kde=False, bins=100)
plt.show()

In [ ]:
adata_v

In [ ]:
import sys
sys.version